In [906]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from urllib.request import urljoin
import time
import random
import copy
import pickle
import sys

In [23]:
#url = 'https://www.imdb.com/search/keyword/?mode=advanced&page=1&ref_=kw_nxt&sort=year,desc&title_type=movie&genres=Sci-Fi&release_date=1979%2C2019'
url = 'https://www.imdb.com/search/keyword/?mode=advanced&page=1&ref_=kw_ref_yr&title_type=movie&release_date=1989%2C2019&sort=year,asc&num_votes=10000%2C'
response = requests.get(url)

In [24]:
response.status_code  #200 = success!

200

In [39]:
# def read_url(url):
#     response = requests.get(url)
#     page=response.text
#     soup=BeautifulSoup(page,'lxml')
#     return soup

In [232]:
# i_scores = []

# for url in url_list:
#     scores = read_url(url).find_all("div", {"class": "lister-item-content"})
#     for p in scores:
#         try:
#             if p.find('div')['class'] == ['ratings-bar']:
#                 i_scores.append(float(p.find('div',{'class':'ratings-bar'}).find('strong').get_text()))
#         except:
#             i_scores.append(np.nan)

In [907]:
from requests import get

#url = 'https://www.imdb.com/search/keyword/?mode=advanced&page=1&ref_=kw_nxt&sort=year,desc&title_type=movie&genres=Sci-Fi&release_date=1979%2C2019'
url = 'https://www.imdb.com/search/keyword/?mode=advanced&page=1&ref_=kw_ref_yr&title_type=movie&release_date=1989%2C2019&sort=year,asc&num_votes=10000%2C'
response = get(url)

In [908]:
soup = BeautifulSoup(response.text, 'lxml')

In [909]:
movie_containers = soup.find_all('div', class_ = 'lister-item mode-advanced')
print(type(movie_containers))
print(len(movie_containers))

<class 'bs4.element.ResultSet'>
50


In [910]:
#movie_containers.find('div', class_='lister-item-content')
first_movie = soup.find('div', class_='lister-item mode-advanced')

In [911]:
first_movie.div

<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt0097815"></div>
</div>

In [353]:
first_name = first_movie.h3.a.text

In [354]:
first_year = first_movie.h3.find('span', class_ = 'lister-item-year text-muted unbold').text

In [355]:
first_imdb = float(first_movie.strong.text)

In [356]:
first_metascore = first_movie.find('span', class_ = 'metascore favorable')
first_metascore = int(first_metascore.text)

In [358]:
first_votes = first_movie.find('span', attrs = {'name':'nv'})
first_votes = int(first_votes['data-value'])

In [ ]:
first_runtime = first_movie.find('p', class_='text-muted').find('span', class_='runtime').text

In [517]:
first_certificate = first_movie.find('p', class_='text-muted').find('span', class_='certificate').text

'TV-MA'

In [541]:
first_genres = first_movie.find('p', class_='text-muted').find('span', class_='genre').text.split()

['\nHorror', ' Sci-Fi', ' Thriller            ']

In [442]:
first_director = first_movie.find('div', class_='lister-item-content').find('p', class_="").a.text # picks up only one director
first_director

'Galder Gaztelu-Urrutia'

In [461]:
#first_stars = first_movie.find('div', class_='lister-item-content').find('p', class_="").find_all()
first_stars = first_movie.find('div', class_='lister-item-content').find('p', class_="").find_all('a')[1:] # may pick up any co-directors
first_stars = [a.text for a in first_stars]
first_stars

['Ivan Massagué', 'Zorion Eguileor', 'Antonia San Juan', 'Emilio Buale']

In [912]:
names = []
years = []
imdb_ratings = []
metascores = []
votes = []
runtimes = []
certificates = []
genres = []
directors = []
stars = []

In [913]:
#url = 'https://www.imdb.com/search/keyword/?mode=advanced&page=1&ref_=kw_nxt&sort=year,desc&title_type=movie&genres=Sci-Fi&release_date=1979%2C2019'
url = 'https://www.imdb.com/search/keyword/?mode=advanced&page=1&ref_=kw_ref_yr&title_type=movie&release_date=1989%2C2019&sort=year,asc&num_votes=10000%2C'
response = get(url)
soup = BeautifulSoup(response.text, 'lxml')
movie_containers = soup.find_all('div', class_ = 'lister-item mode-advanced')

for container in movie_containers:
    # If the movie has Metascore, then extract:
    if container.find('div', class_ = 'ratings-metascore') is not None: # or container.find('span', attrs = {'name':'nv'})['data-value'] is not None:
        name = container.h3.a.text
        names.append(name)
        year = container.h3.find('span', class_ = 'lister-item-year').text
        years.append(year)
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        m_score = container.find('span', class_ = 'metascore').text
        metascores.append(int(m_score))
        vote = container.find('span', attrs = {'name':'nv'})['data-value']
        votes.append(int(vote))
        
        runtime = container.find('p', class_='text-muted').find('span', class_='runtime').text
        runtimes.append(runtime)
        
        certificate = container.find('p', class_='text-muted').find('span', class_='certificate').text
        certificates.append(certificate)
        
        genre = container.find('p', class_='text-muted').find('span', class_='genre').text.split()
        genres.append(genre)
        
        director = container.find('div', class_='lister-item-content').find('p', class_="").a.text # picks up only one director
        directors.append(director)
        star = container.find('div', class_='lister-item-content').find('p', class_="").find_all('a')[1:] # may pick up any co-directors
        star = [a.text for a in star]   
        stars.append(star)

In [914]:
test_df = pd.DataFrame({'movie': names,
'year': years,
'imdb': imdb_ratings,
'metascore': metascores,
'votes': votes,
'runtime': runtimes,
'certificate': certificates,
'genre': genres,
'director': directors,
'stars': stars                        
})
print(test_df.info())
test_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie        48 non-null     object 
 1   year         48 non-null     object 
 2   imdb         48 non-null     float64
 3   metascore    48 non-null     int64  
 4   votes        48 non-null     int64  
 5   runtime      48 non-null     object 
 6   certificate  48 non-null     object 
 7   genre        48 non-null     object 
 8   director     48 non-null     object 
 9   stars        48 non-null     object 
dtypes: float64(1), int64(2), object(7)
memory usage: 3.9+ KB
None


,movie,year,imdb,metascore,votes,runtime,certificate,genre,director,stars
0,Major League,(1989),7.2,62,59421,107 min,R,"[Comedy,, Sport]",David S. Ward,"[Tom Berenger, Charlie Sheen, Corbin Bernsen, ..."
1,Indiana Jones and the Last Crusade,(1989),8.2,65,662755,127 min,PG-13,"[Action,, Adventure]",Steven Spielberg,"[Harrison Ford, Sean Connery, Alison Doody, De..."
2,Licence to Kill,(1989),6.6,58,90114,133 min,PG-13,"[Action,, Adventure,, Thriller]",John Glen,"[Timothy Dalton, Robert Davi, Carey Lowell, Ta..."
3,Dead Poets Society,(1989),8.1,79,397339,128 min,PG,"[Comedy,, Drama]",Peter Weir,"[Robin Williams, Robert Sean Leonard, Ethan Ha..."
4,Batman,(1989),7.5,69,326245,126 min,PG-13,"[Action,, Adventure]",Tim Burton,"[Michael Keaton, Jack Nicholson, Kim Basinger,..."
5,"Honey, I Shrunk the Kids",(1989),6.3,63,136578,93 min,PG,"[Adventure,, Comedy,, Family]",Joe Johnston,"[Rick Moranis, Matt Frewer, Marcia Strassman, ..."
6,Back to the Future Part II,(1989),7.8,57,452459,108 min,PG,"[Adventure,, Comedy,, Sci-Fi]",Robert Zemeckis,"[Michael J. Fox, Christopher Lloyd, Lea Thomps..."
7,The Little Mermaid,(1989),7.6,88,226241,83 min,G,"[Animation,, Family,, Fantasy]",Ron Clements,"[John Musker, Jodi Benson, Samuel E. Wright, R..."
8,Road House,(1989),6.6,36,59245,114 min,R,"[Action,, Thriller]",Rowdy Herrington,"[Patrick Swayze, Kelly Lynch, Sam Elliott, Ben..."
9,Heathers,(1989),7.2,72,83394,103 min,R,"[Comedy,, Crime]",Michael Lehmann,"[Winona Ryder, Christian Slater, Shannen Doher..."


In [915]:
#url = 'https://www.imdb.com/search/keyword/?mode=advanced&page=1&ref_=kw_nxt&sort=year,desc&title_type=movie&genres=Sci-Fi&release_date=1979%2C2019'
url = 'https://www.imdb.com/search/keyword/?mode=advanced&page=1&ref_=kw_ref_yr&title_type=movie&release_date=1989%2C2019&sort=year,asc&num_votes=10000%2C'

In [916]:
#pages = [str(i) for i in range(1, 161)] 
pages = [str(i) for i in range(1, 139)] #138 pages 

In [413]:
from warnings import warn

warn("Warning Simulation")

/Users/johnlee/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Warning Simulation
  This is separate from the ipykernel package so we can avoid doing imports until


In [468]:
import time

In [918]:
#pages = [str(i) for i in range(1, 161)] 
pages = [str(i) for i in range(1, 139)] #138 pages 

In [919]:
names = []
years = []
imdb_ratings = []
metascores = []
votes = []
runtimes = []
certificates = []
genres = []
directors = []
stars = []

start_time = time.time()
requests = 0

for page in pages:

    # Make a get request
    response = get('https://www.imdb.com/search/keyword/?mode=advanced&page=' + page +  
                   '&ref_=kw_nxt&title_type=movie&release_date=1989%2C2019&sort=year,asc&num_votes=10000%2C')
    
    # Pause the loop
    time.sleep(random.randint(2,8))

    # Monitor the requests
    requests += 1
    elapsed_time = time.time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    # Break the loop if the number of requests is greater than expected
    if requests > 200:
        warn('Number of requests was greater than expected.')
        break

    # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(response.text, 'lxml') # html.parser

    # Select all the 50 movie containers from a single page
    movie_containers = soup.find_all('div', class_ = 'lister-item mode-advanced')

    # For every movie of these 50
    for container in movie_containers:
        # If the movie has a Metascore, then:
        #if container.find('div', class_ = 'ratings-metascore') is not None:

            # Scrape the name
            name = container.h3.a.text
            names.append(name)

            # Scrape the year
            year = container.h3.find('span', class_ = 'lister-item-year').text
            years.append(year)
            
            # Scrape the IMDB rating
            imdb = container.strong
            if imdb:
                imdb_ratings.append(float(imdb.text))
            else:
                imdb = 'no stars given'
                imdb_ratings.append(imdb)

            # Scrape the Metascore
            m_score = container.find('span', class_ = 'metascore')
            if m_score:
                metascores.append(int(m_score.text))
            else:
                m_score = 'no metascore'
                metascores.append(m_score)

            # Scrape the number of votes
            vote = container.find('span', attrs = {'name':'nv'})
            if vote:
                vote = vote['data-value']
                votes.append(int(vote))
            else:
                vote = 'no votes'
                votes.append(vote)
            
            # Scrape the runtime
            runtime = container.find('p', class_='text-muted').find('span', class_='runtime')
            if runtime:
                runtimes.append(runtime.text)
            else:
                runtime = 'no runtime'
                runtimes.append(runtime)

            # Scrape the certificate            
            certificate = container.find('p', class_='text-muted').find('span', class_='certificate')
            #certificate = container.find('p', class_='text-muted').find('span', class_='certificate').text
            if certificate:
                certificates.append(certificate.text)
            else:
                certificate = 'not rated'
                certificates.append(certificate)

            # Scrape the genre
            genre = container.find('p', class_='text-muted').find('span', class_='genre')
            if genre:
                genres.append(genre.text.split())
            else:
                genre = 'no genre'
                genres.append(genre)

            # Scrape the name of the director
            director = container.find('div', class_='lister-item-content').find('p', class_="").a # picks up only one director
            if director:
                directors.append(director.text)
            else:
                director = 'no director'
                directors.append(director)

            # Scrape the names of the stars
            star = container.find('div', class_='lister-item-content').find('p', class_="").find_all('a') # may pick up any co-directors
            if star:
                star = star[1:]
                star = [a.text for a in star]   
                stars.append(star)
            else:            
                star = 'no stars'
                stars.append(star)                                    
        
print((time.time()-start_time)/60, "minutes")


15.770182899634044 minutes


In [921]:
imdb_data = pd.DataFrame({'movie': names,
'year': years,
'imdb': imdb_ratings,
'metascore': metascores,
'votes': votes,
'runtime': runtimes,
'certificate': certificates,
'genre': genres,                          
'director': directors,
'stars': stars                    
})

print(imdb_data.info())
imdb_data.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6864 entries, 0 to 6863
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie        6864 non-null   object 
 1   year         6864 non-null   object 
 2   imdb         6864 non-null   float64
 3   metascore    6864 non-null   object 
 4   votes        6864 non-null   int64  
 5   runtime      6864 non-null   object 
 6   certificate  6864 non-null   object 
 7   genre        6864 non-null   object 
 8   director     6864 non-null   object 
 9   stars        6864 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 536.4+ KB
None


,movie,year,imdb,metascore,votes,runtime,certificate,genre,director,stars
3143,No Reservations,(2007),6.3,50,68500,104 min,PG,"[Comedy,, Drama,, Romance]",Scott Hicks,"[Catherine Zeta-Jones, Aaron Eckhart, Abigail ..."
3419,The Secret Life of Bees,(2008),7.3,57,23354,114 min,PG-13,[Drama],Gina Prince-Bythewood,"[Dakota Fanning, Jennifer Hudson, Queen Latifa..."
2631,Must Love Dogs,(2005),5.9,46,29843,98 min,PG-13,"[Comedy,, Romance]",Gary David Goldberg,"[Diane Lane, John Cusack, Elizabeth Perkins, C..."
1321,A Midsummer Night's Dream,(1999),6.4,61,23814,116 min,PG-13,"[Comedy,, Fantasy,, Romance]",Michael Hoffman,"[Kevin Kline, Michelle Pfeiffer, Rupert Everet..."
1615,Lara Croft: Tomb Raider,(2001),5.8,33,192151,100 min,PG-13,"[Action,, Adventure,, Fantasy]",Simon West,"[Angelina Jolie, Jon Voight, Iain Glen, Noah T..."
2512,Cinderella Man,(2005),8.0,69,170969,144 min,PG-13,"[Biography,, Drama,, History]",Ron Howard,"[Russell Crowe, Renée Zellweger, Craig Bierko,..."
4646,Jab Tak Hai Jaan,(2012),6.7,no metascore,49575,176 min,Not Rated,"[Drama,, Romance]",Yash Chopra,"[Shah Rukh Khan, Katrina Kaif, Anushka Sharma,..."
2209,Howl's Moving Castle,(2004),8.2,80,302834,119 min,PG,"[Animation,, Adventure,, Family]",Hayao Miyazaki,"[Chieko Baishô, Takuya Kimura, Tatsuya Gashûin..."
370,Radio Flyer,(1992),7.0,no metascore,11953,114 min,PG-13,[Drama],Richard Donner,"[David Mickey Evans, Lorraine Bracco, John Hea..."
4920,21 & Over,(2013),5.8,34,70444,93 min,R,[Comedy],Jon Lucas,"[Scott Moore, Miles Teller, Justin Chon, Jonat..."


In [923]:
imdb_data.to_pickle('imdb_scraped_data.pkl')  # where to save it, usually as a .pkl

In [ ]:
imdb_data = pd.read_pickle('imdb_scraped_data.pkl') # load the dataframe

In [924]:
imdb_data.sort_values(by='votes',ascending=False).head()

,movie,year,imdb,metascore,votes,runtime,certificate,genre,director,stars
515,The Shawshank Redemption,(1994),9.3,80,2216839,142 min,R,[Drama],Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
3298,The Dark Knight,(2008),9.0,84,2193484,152 min,PG-13,"[Action,, Crime,, Drama]",Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
3899,Inception,(2010),8.8,74,1943744,148 min,PG-13,"[Action,, Adventure,, Sci-Fi]",Christopher Nolan,"[Leonardo DiCaprio, Joseph Gordon-Levitt, Elle..."
1231,Fight Club,(1999),8.8,66,1768050,139 min,R,[Drama],David Fincher,"[Brad Pitt, Edward Norton, Meat Loaf, Zach Gre..."
516,Pulp Fiction,(1994),8.9,94,1739947,154 min,R,"[Crime,, Drama]",Quentin Tarantino,"[John Travolta, Uma Thurman, Samuel L. Jackson..."


Consider engineering the following features:
* Number of genres 
* Length of movie title
* Number of male/female stars (or ratio)



Now scrape in domestic sales from the-numbers site

In [578]:
url = 'http://www.the-numbers.com/movie/budgets/all'
response = get(url)
soup = BeautifulSoup(response.text, 'lxml')

In [585]:
movie_table = soup.find('table')
print(type(movie_tables))
print(len(movie_tables))

<class 'bs4.element.Tag'>
203


In [586]:
print(movie_table.prettify())

<table>
 <tr>
  <th>
  </th>
  <th>
   Release
   <br/>
   Date
  </th>
  <th>
   Movie
  </th>
  <th>
   Production
   <br/>
   Budget
  </th>
  <th>
   Domestic
   <br/>
   Gross
  </th>
  <th>
   Worldwide
   <br/>
   Gross
  </th>
 </tr>
 <tr>
  <td class="data">
   1
  </td>
  <td>
   <a href="/box-office-chart/daily/2019/04/23">
    Apr 23, 2019
   </a>
  </td>
  <td>
   <b>
    <a href="/movie/Avengers-Endgame-(2019)#tab=summary">
     Avengers: Endgame
    </a>
   </b>
  </td>
  <td class="data">
   $400,000,000
  </td>
  <td class="data">
   $858,373,000
  </td>
  <td class="data">
   $2,797,800,564
  </td>
 </tr>
 <tr>
  <td class="data">
   2
  </td>
  <td>
   <a href="/box-office-chart/daily/2011/05/20">
    May 20, 2011
   </a>
  </td>
  <td>
   <b>
    <a href="/movie/Pirates-of-the-Caribbean-On-Stranger-Tides#tab=summary">
     Pirates of the Caribbean: On Stranger Tides
    </a>
   </b>
  </td>
  <td class="data">
   $379,000,000
  </td>
  <td class="data">
   $241,063,

In [587]:
rows = [row for row in movie_table.find_all('tr')]  # tr tag is for rows

In [598]:
rows[1:3]

[<tr><td class="data">1</td>
 <td><a href="/box-office-chart/daily/2019/04/23">Apr 23, 2019</a></td>
 <td><b><a href="/movie/Avengers-Endgame-(2019)#tab=summary">Avengers: Endgame</a></b></td>
 <td class="data"> $400,000,000</td>
 <td class="data"> $858,373,000</td>
 <td class="data"> $2,797,800,564</td>
 </tr>,
 <tr><td class="data">2</td>
 <td><a href="/box-office-chart/daily/2011/05/20">May 20, 2011</a></td>
 <td><b><a href="/movie/Pirates-of-the-Caribbean-On-Stranger-Tides#tab=summary">Pirates of the Caribbean: On Stranger Tides</a></b></td>
 <td class="data"> $379,000,000</td>
 <td class="data"> $241,063,875</td>
 <td class="data"> $1,045,663,875</td>
 </tr>]

In [634]:
movies = {}

for row in rows[1:6]:
    items = row.find_all('td')
#     row_index = items[0]
#     release_date = items[1].find('a').text
    title = items[2].find('a').text
#     production_budget = items[3].text
#     domestic_gross = items[4].text
#     worldwide_gross = items[5].text
    movies[title] = [i.text for i in items]
    
movies

{'Avengers: Endgame': ['1',
  'Apr 23, 2019',
  'Avengers: Endgame',
  '\xa0$400,000,000',
  '\xa0$858,373,000',
  '\xa0$2,797,800,564'],
 'Pirates of the Caribbean: On Stranger Tides': ['2',
  'May 20, 2011',
  'Pirates of the Caribbean: On Stranger Tides',
  '\xa0$379,000,000',
  '\xa0$241,063,875',
  '\xa0$1,045,663,875'],
 'Avengers: Age of Ultron': ['3',
  'Apr 22, 2015',
  'Avengers: Age of Ultron',
  '\xa0$365,000,000',
  '\xa0$459,005,868',
  '\xa0$1,403,013,963'],
 'Star Wars Ep. VII: The Force Awakens': ['4',
  'Dec 16, 2015',
  'Star Wars Ep. VII: The Force Awakens',
  '\xa0$306,000,000',
  '\xa0$936,662,225',
  '\xa0$2,068,223,624'],
 'Avengers: Infinity War': ['5',
  'Apr 25, 2018',
  'Avengers: Infinity War',
  '\xa0$300,000,000',
  '\xa0$678,815,482',
  '\xa0$2,048,359,754']}

In [637]:
movies['Avengers: Infinity War']

['5',
 'Apr 25, 2018',
 'Avengers: Infinity War',
 '\xa0$300,000,000',
 '\xa0$678,815,482',
 '\xa0$2,048,359,754']

In [689]:
pages = ['/' + str(i) for i in np.arange(101, 6002, 100)] #61 pages of results
pages.insert(0, "")

#for page in pages:

    # Make a get request
 #   response = get('https://www.the-numbers.com/movie/budgets/all' + page)

In [690]:
pages

['',
 '/101',
 '/201',
 '/301',
 '/401',
 '/501',
 '/601',
 '/701',
 '/801',
 '/901',
 '/1001',
 '/1101',
 '/1201',
 '/1301',
 '/1401',
 '/1501',
 '/1601',
 '/1701',
 '/1801',
 '/1901',
 '/2001',
 '/2101',
 '/2201',
 '/2301',
 '/2401',
 '/2501',
 '/2601',
 '/2701',
 '/2801',
 '/2901',
 '/3001',
 '/3101',
 '/3201',
 '/3301',
 '/3401',
 '/3501',
 '/3601',
 '/3701',
 '/3801',
 '/3901',
 '/4001',
 '/4101',
 '/4201',
 '/4301',
 '/4401',
 '/4501',
 '/4601',
 '/4701',
 '/4801',
 '/4901',
 '/5001',
 '/5101',
 '/5201',
 '/5301',
 '/5401',
 '/5501',
 '/5601',
 '/5701',
 '/5801',
 '/5901',
 '/6001']

In [691]:
for page in pages:
    url = 'https://www.the-numbers.com/movie/budgets/all'
    var = url + page
    print(var)

https://www.the-numbers.com/movie/budgets/all
https://www.the-numbers.com/movie/budgets/all/101
https://www.the-numbers.com/movie/budgets/all/201
https://www.the-numbers.com/movie/budgets/all/301
https://www.the-numbers.com/movie/budgets/all/401
https://www.the-numbers.com/movie/budgets/all/501
https://www.the-numbers.com/movie/budgets/all/601
https://www.the-numbers.com/movie/budgets/all/701
https://www.the-numbers.com/movie/budgets/all/801
https://www.the-numbers.com/movie/budgets/all/901
https://www.the-numbers.com/movie/budgets/all/1001
https://www.the-numbers.com/movie/budgets/all/1101
https://www.the-numbers.com/movie/budgets/all/1201
https://www.the-numbers.com/movie/budgets/all/1301
https://www.the-numbers.com/movie/budgets/all/1401
https://www.the-numbers.com/movie/budgets/all/1501
https://www.the-numbers.com/movie/budgets/all/1601
https://www.the-numbers.com/movie/budgets/all/1701
https://www.the-numbers.com/movie/budgets/all/1801
https://www.the-numbers.com/movie/budgets/all

In [925]:
movie_names = []
release_dates = []
production_budgets = []
domestic_grosses = []
worldwide_grosses = []

start_time = time.time()
requests = 0

pages = ['/' + str(i) for i in np.arange(101, 6002, 100)] #61 pages of results
pages.insert(0, "") #insert "" in front of list

for page in pages:

    # Make a get request
    url = 'https://www.the-numbers.com/movie/budgets/all'
    response = get(url + page)
    
    # Pause the loop
    time.sleep(random.randint(2,8))

    # Monitor the requests
    requests += 1
    elapsed_time = time.time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    # Break the loop if the number of requests is greater than expected
    if requests > 200:
        warn('Number of requests was greater than expected.')
        break

    # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(response.text, 'lxml') 
    movie_table = soup.find('table')
    rows = [row for row in movie_table.find_all('tr')]
    
    for row in rows[1:101]:
        items = row.find_all('td')

        # Scrape the release date
        release_date = items[1].find('a')
        if release_date:
            release_dates.append(release_date.text)
        else:
            release_date = 'no release date'
            release_dates.append(release_date)

        # Scrape the movie name
        movie_name = items[2].find('a')
        if movie_name:
            movie_names.append(movie_name.text)
        else:
            movie_name = 'no movie name'
            movie_names.append(movie_name)

        # Scrape the production budget
        production_budget = items[3]
        if production_budget:
            production_budgets.append(production_budget.text)
        else:
            production_budget = 'no budget'
            production_budgets.append(production_budget)

        # Scrape the domestic gross
        domestic_gross = items[4]
        if domestic_gross:
            domestic_grosses.append(domestic_gross.text)
        else:
            domestic_gross = 'no domestic gross'
            domestic_grosses.append(domestic_gross)  

        # Scrape the worldwide gross
        worldwide_gross = items[5]
        if worldwide_gross:
            worldwide_grosses.append(worldwide_gross.text)
        else:
            worldwide_gross = 'no worldwide gross'
            worldwide_grosses.append(worldwide_gross)  

print((time.time()-start_time)/60, "minutes")

6.228843434651693 minutes


In [926]:
len(movie_names)

6027

In [927]:
thenumbers_data = pd.DataFrame({'movie': movie_names,
'release_date': release_dates,
'production_budget': production_budgets,
'domestic_gross': domestic_grosses,
'worldwide_gross': worldwide_grosses                    
})

print(thenumbers_data.info())
thenumbers_data.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6027 entries, 0 to 6026
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   movie              6027 non-null   object
 1   release_date       6027 non-null   object
 2   production_budget  6027 non-null   object
 3   domestic_gross     6027 non-null   object
 4   worldwide_gross    6027 non-null   object
dtypes: object(5)
memory usage: 235.6+ KB
None


,movie,release_date,production_budget,domestic_gross,worldwide_gross
1795,Punisher: War Zone,"Dec 5, 2008","$35,000,000","$8,050,977","$10,157,534"
2173,Spy Kids: All the Time in the World,"Aug 19, 2011","$27,000,000","$38,536,376","$80,681,183"
1150,Christmas with the Kranks,"Nov 24, 2004","$50,000,000","$73,701,902","$96,469,187"
3430,The Killer Inside Me,"Jun 18, 2010","$13,000,000","$217,277","$3,617,277"
3469,The Golden Child,"Dec 12, 1986","$12,000,000","$79,817,937","$79,817,937"
5130,The Florida Project,"Oct 6, 2017","$2,000,000","$5,904,366","$11,295,324"
335,San Andreas,"May 27, 2015","$110,000,000","$155,190,832","$456,258,539"
4585,Yoga Hosers,"Jul 29, 2016","$5,000,000",$0,"$22,958"
5357,San Francisco,"Jun 26, 1936","$1,300,000","$2,868,000","$5,273,000"
2004,Dumb and Dumberer: When Harry Met Lloyd,"Jun 13, 2003","$30,000,000","$26,214,846","$26,292,856"


In [929]:
thenumbers_data.to_pickle('thenumbers_scraped_data.pkl')

In [928]:
thenumbers_data

,movie,release_date,production_budget,domestic_gross,worldwide_gross
0,Avengers: Endgame,"Apr 23, 2019","$400,000,000","$858,373,000","$2,797,800,564"
1,Pirates of the Caribbean: On Stranger Tides,"May 20, 2011","$379,000,000","$241,063,875","$1,045,663,875"
2,Avengers: Age of Ultron,"Apr 22, 2015","$365,000,000","$459,005,868","$1,403,013,963"
3,Star Wars Ep. VII: The Force Awakens,"Dec 16, 2015","$306,000,000","$936,662,225","$2,068,223,624"
4,Avengers: Infinity War,"Apr 25, 2018","$300,000,000","$678,815,482","$2,048,359,754"
...,...,...,...,...,...
6022,Red 11,Unknown,"$7,000",$0,$0
6023,Following,"Apr 2, 1999","$6,000","$48,482","$240,495"
6024,Return to the Land of Wonders,"Jul 13, 2005","$5,000","$1,338","$1,338"
6025,A Plague So Pleasant,"Sep 29, 2015","$1,400",$0,$0


Now scrape top 1000 popular actors from imdb (starmeter)

In [762]:
url = 'https://www.imdb.com/search/name/?gender=male,female&ref_=rlm'
response = get(url)
soup = BeautifulSoup(response.text, 'lxml')

In [763]:
pages = []
pages.append(url)

In [764]:
for i in np.arange(51, 952, 50):
    i = str(i)
    page = 'https://www.imdb.com/search/name/?gender=male,female&start=' + i + '&ref_=rlm'
    pages.append(page)

In [765]:
pages

['https://www.imdb.com/search/name/?gender=male,female&ref_=rlm',
 'https://www.imdb.com/search/name/?gender=male,female&start=51&ref_=rlm',
 'https://www.imdb.com/search/name/?gender=male,female&start=101&ref_=rlm',
 'https://www.imdb.com/search/name/?gender=male,female&start=151&ref_=rlm',
 'https://www.imdb.com/search/name/?gender=male,female&start=201&ref_=rlm',
 'https://www.imdb.com/search/name/?gender=male,female&start=251&ref_=rlm',
 'https://www.imdb.com/search/name/?gender=male,female&start=301&ref_=rlm',
 'https://www.imdb.com/search/name/?gender=male,female&start=351&ref_=rlm',
 'https://www.imdb.com/search/name/?gender=male,female&start=401&ref_=rlm',
 'https://www.imdb.com/search/name/?gender=male,female&start=451&ref_=rlm',
 'https://www.imdb.com/search/name/?gender=male,female&start=501&ref_=rlm',
 'https://www.imdb.com/search/name/?gender=male,female&start=551&ref_=rlm',
 'https://www.imdb.com/search/name/?gender=male,female&start=601&ref_=rlm',
 'https://www.imdb.com/

In [766]:
containers = soup.find('div', class_='lister-list')

In [770]:
print(containers.prettify())

<div class="lister-list">
 <div class="lister-item mode-detail">
  <div class="lister-item-image">
   <a href="/name/nm3400186">
    <img alt="Julia Garner" height="209" src="https://m.media-amazon.com/images/M/MV5BNmU3NDRjMDEtNTQ2Ni00ZGZmLTk3ZjUtODUxMjA2NGY4NDg4XkEyXkFqcGdeQXVyMjQzMDM0NDU@._V1_UX140_CR0,0,140,209_AL_.jpg" width="140"/>
   </a>
  </div>
  <div class="lister-item-content">
   <h3 class="lister-item-header">
    <span class="lister-item-index unbold text-primary">
     1.
    </span>
    <a href="/name/nm3400186">
     Julia Garner
    </a>
   </h3>
   <p class="text-muted text-small">
    Actress
    <span class="ghost">
     |
    </span>
    <a href="/title/tt1441326/">
     Martha Marcy May Marlene
    </a>
   </p>
   <p>
    Julia Garner is an American actress and model. She is best known for her roles in Electrick Children (2012), We Are What We Are (2013), The Perks of Being a Wallflower (2012), Sin City: A Dame to Kill For (2014), and Grandma (2015).
    <br/>
  

In [ ]:
star_name = containers.find('img', alt=True)['alt']

In [772]:
star_ranking = containers.find('span', class_="lister-item-index").text

'1. '

In [809]:
actor_or_actress = containers.find('p', class_='text-muted').find(text=True, recursive=False)

'\n                         Actress '

In [901]:
star_names = []
star_rankings = []
actor_or_actresses = []

start_time = time.time()
requests = 0

pages = []
first_page = 'https://www.imdb.com/search/name/?gender=male,female&ref_=rlm'
pages.append(first_page)
pagerange = [str(i) for i in np.arange(51, 952, 50)] #51, 952, 50 

for page in pagerange: 
    page = 'https://www.imdb.com/search/name/?gender=male,female&start=' + page + '&ref_=rlm'                                   
    pages.append(page)

for page in pages:

    # Make a get request
    #url = 'https://www.imdb.com/search/name/?gender=male,female&ref_=rlm'
    response = get(page)
    
    # Pause the loop
    time.sleep(random.randint(2,8))

    # Monitor the requests
    requests += 1
    elapsed_time = time.time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    # Break the loop if the number of requests is greater than expected
    if requests > 200:
        warn('Number of requests was greater than expected.')
        break

    # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(response.text, 'lxml') 
    containers = soup.find_all('div', class_='lister-item')
    
    for container in containers:
        # Scrape the star's name
        star_name = container.find('img', alt=True)['alt']
        if star_name:
            star_names.append(star_name)
        else:
            star_name = 'no name'
            star_names.append(star_name)

        # Scrape the star's ranking
        star_ranking = container.find('span', class_="lister-item-index")
        if star_ranking:
            star_rankings.append(star_ranking.text)
        else:
            star_ranking = 'no ranking'
            star_rankings.append(star_ranking)

        # Scrape if star is actor or actress
        actor_or_actress = container.find('p', class_='text-muted').find(text=True, recursive=False)    
        if actor_or_actress:
            actor_or_actresses.append(actor_or_actress)
        else:
            actor_or_actress = 'no type'
            actor_or_actresses.append(actor_or_actress)
 
print((time.time()-start_time)/60, "minutes")

2.0943224668502807 minutes


In [903]:
star_ranking_data = pd.DataFrame({'star_name': star_names,
'star_ranking': star_rankings,
'actor_or_actress': actor_or_actresses,
})

print(star_ranking_data.info())
star_ranking_data.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   star_name         1000 non-null   object
 1   star_ranking      1000 non-null   object
 2   actor_or_actress  1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB
None


,star_name,star_ranking,actor_or_actress
944,Haley Lu Richardson,945.,\n Actress
479,Aras Bulut Iynemli,480.,\n Actor
513,Tobias Menzies,514.,\n Actor
444,Andrew Scott,445.,\n Actor
440,Ivana Milicevic,441.,\n Actress
861,Drew Barrymore,862.,\n Producer
558,Larisa Oleynik,559.,\n Actress
510,Essie Davis,511.,\n Actress
618,Laura Harrier,619.,\n Actress
65,Henry Cavill,66.,\n Actor


In [904]:
sys.setrecursionlimit(10000)

In [905]:
star_ranking_data.to_pickle('star_scraped_data.pkl')